In [4]:
import numpy as np
import pandas as pd
import os
import sys

# Add the directory containing the VotingModel to the Python path
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/'))
from util.voting_rules import mean_aggregation, median_aggregation, quadratic_aggregation
from model.VotingModel import VotingModel

def random_change_vote(vote, num_projects):
    """Randomly change a single vote."""
    new_vote = vote.copy()
    change_idx = np.random.randint(0, num_projects)
    new_vote[change_idx] = np.random.uniform(0, 1)
    return new_vote

def calculate_distance(x, x_prime):
    """Calculate the distance between two outcomes using L2 distance."""
    return np.linalg.norm(x - x_prime)

def evaluate_robustness(model, num_iterations):
    robustness_results = []
    for method in ["mean", "median", "quadratic"]:
        distances = []
        for _ in range(num_iterations):
            # Step 1: Take the original vote profile and compute the outcome
            original_outcome = model.allocate_funds(method)

            # Step 2: Randomly change the vote of a single voter
            voter_idx = np.random.randint(0, model.num_voters)
            original_vote = model.voting_matrix[voter_idx].copy()
            new_vote = random_change_vote(original_vote, model.num_projects)
            new_voting_matrix = model.voting_matrix.copy()
            new_voting_matrix[voter_idx] = new_vote

            # Update the model's voting matrix
            model.voting_matrix = new_voting_matrix

            # Step 3: Compute the new outcome
            new_outcome = model.allocate_funds(method)

            # Step 4: Calculate the distance between the original and new outcomes
            distance = calculate_distance(original_outcome, new_outcome)
            distances.append(distance)

            # Reset the voting matrix to the original
            model.voting_matrix[voter_idx] = original_vote

        # Step 5: Compute the average distance
        average_distance = np.mean(distances)
        robustness_results.append({
            "voting_rule": method,
            "average_distance": average_distance
        })

    return pd.DataFrame(robustness_results)

# Initialize the model
num_voters = 144
num_projects = 600
total_op_tokens = 30e6
rounds=1000
model = VotingModel(num_voters=num_voters, num_projects=num_projects, total_op_tokens=total_op_tokens)
model.step()

# Evaluate robustness
robustness_results = evaluate_robustness(model,rounds)
robustness_results



/Users/idrees/Code/govxs/agents/VoterAgent.py:8: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


,voting_rule,average_distance
0,mean,347.274596
1,median,494.432460
2,quadratic,703.083374
